In [1]:
import numpy as np

from lightfm.datasets import fetch_movielens

movielens = fetch_movielens()

In [2]:
for key, value in movielens.items():
    print(key, type(value), value.shape)

('test', <class 'scipy.sparse.coo.coo_matrix'>, (943, 1682))
('item_features', <class 'scipy.sparse.csr.csr_matrix'>, (1682, 1682))
('train', <class 'scipy.sparse.coo.coo_matrix'>, (943, 1682))
('item_labels', <type 'numpy.ndarray'>, (1682,))
('item_feature_labels', <type 'numpy.ndarray'>, (1682,))


In [3]:
train = movielens['train']
test = movielens['test']

In [42]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM(learning_rate=0.05, loss='warp', no_components=32, item_alpha=0.0001)

model.fit_partial(train, item_features=movielens['item_features'], epochs=20 )

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.70, test 0.10.
AUC: train 0.96, test 0.91.


In [43]:
item_vectors = movielens['item_features'] * model.item_embeddings

In [52]:
from annoy import AnnoyIndex
f = 32
t = AnnoyIndex(f)  # Length of item vector that will be indexed
for i in xrange(item_vectors.shape[0]):
    v = item_vectors[i]
    t.add_item(i, v)

t.build(32) # 5 trees
t.save('test.ann')

True

In [58]:
def nearest_movies(movie_id, index, n=10):
    nn = index.get_nns_by_item(movie_id, 10)
    print('Closest to %s : \n' % movielens['item_labels'][movie_id])
    titles = [movielens['item_labels'][i] for i in nn]
    print("\n".join(titles))

In [59]:
nearest_movies(90, t)

Closest to Nightmare Before Christmas, The (1993) : 

Nightmare Before Christmas, The (1993)
Beauty and the Beast (1991)
Fantasia (1940)
Heavy Metal (1981)
E.T. the Extra-Terrestrial (1982)
Aladdin (1992)
Raiders of the Lost Ark (1981)
Lion King, The (1994)
Princess Bride, The (1987)
Monty Python and the Holy Grail (1974)


In [61]:
"""
Use a ball tree to find vectors maximising inner product
with a query efficiently.
Based on P. Ram & A. Gray, Maximum Inner-Product Search
using Tree Data-Structures, 2012.

From https://github.com/gamboviol/miptree/blob/master/mip_tree.py
"""

import random
import numpy as np
from operator import itemgetter

def d2(S,x):
    D = S - np.tile(x,(len(S),1))
    return sum((D*D).T)

class Node(object):
    
    def __init__(self,S,indices):
        self.S = indices
        self.mu = np.mean(S[indices],axis=0)
        self.R = max(d2(S[indices],self.mu))**0.5
        self.left = None
        self.right = None

    def dump(self,char,level):
        for x in self.S:
            print '{0}{1}:{2}'.format(' '*level,char,x)
        if self.left:
            self.left.dump('L',level+1)
        if self.right:
            self.right.dump('R',level+1)

    def get_ordering(self,indices):
        if self.left and self.right:
            self.left.get_ordering(indices)
            self.right.get_ordering(indices)
        else:
            indices.extend(self.S)

    def set_ordering(self,index_map):
        self.S = [index_map[ix] for ix in self.S]
        if self.left and self.right:
            self.left.set_ordering(index_map)
            self.right.set_ordering(index_map)

    def scan(self,vecs,q):
        """compute dot products for all the vectors
           indexed by this node"""
        d = [(ix,q.dot(vecs[ix])) for ix in self.S]
        return d

class Query(object):

    def __init__(self,q,k):
        self.q = q
        self.norm = sum(q**2)**0.5
        self.l = -np.inf
        self.k = k
        self.nn = []
        self.scanned = 0

    def dot(self,v):
        return self.q.dot(v)

    def update(self,d):
        self.scanned += len(d)
        self.nn.extend(d)
        self.nn.sort(key=itemgetter(1),reverse=True)
        self.nn = self.nn[:self.k]
        self.l = self.nn[-1][1]             
    
class MIPTree(object):
    """Maximum Inner Product Ball Tree"""

    def __init__(self,S,N0=20,reorder=False):
        """Construct a tree from a dataset
        S        numpy array of data vectors
        N0       the maximum size of each leaf node
        reorder  if True reorder the rows of S to optimise match time,
                 the reordered row indices will be stored in the indices field
        Reordering ensures that data is arranged serially in each leaf node
        which should reduce memory access time - you are only likely to see
        a benefit for very large data arrays.
        """
        self.N0 = N0
        self.root = Node(S,np.arange(len(S)))
        self.build(S,self.root)
        if reorder:
            self.indices = self.get_ordering()
            # now map these to dense range
            index_map = dict((ix,i) for i,ix in enumerate(self.indices))
            self.set_ordering(index_map)
            # indices in each leaf node are arranged serially, reorder the data to match
            S.data = S[self.indices].data
        self.S = S

    def match(self,query,k):
        """Search the tree for vectors maximising inner product with query
        query  the query vector
        k      number of best matches to return
        Returns the matches found as a list of pairs (index into S, inner product),
        and the number of vectors for which the inner product had to be computed.
        """
        q = Query(query,k)
        self.search(q,self.root)
        return q.nn,q.scanned

    def dump(self):
        """Print a primitive visualization of the tree"""
        self.root.dump('',0)

    def make_split(self,S):
        x = random.choice(S)
        A = S[np.argmax(d2(S,x))]
        d2A = d2(S,A)
        B = S[np.argmax(d2A)]
        dd = d2A - d2(S,B)
        return A,B,dd

    def build(self,S,node):
        if len(node.S) > self.N0:
            A,B,dd = self.make_split(S[node.S])
            node.left = Node(S,node.S[dd<=0])
            node.right = Node(S,node.S[dd>0])
            self.build(S,node.left)
            self.build(S,node.right)

    def mip(self,q,node):
        return q.dot(node.mu) + q.norm*node.R

    def search(self,q,node):
        if q.l < self.mip(q,node):
            if node.left is None and node.right is None:
                d = node.scan(self.S,q)
                q.update(d)
            else:
                left = self.mip(q,node.left)
                right = self.mip(q,node.right)
                if q.l < left or q.l < right:
                    if left < right:
                        self.search(q,node.right)
                        if q.l < left:
                            self.search(q,node.left)
                    else:
                        self.search(q,node.left)
                        if q.l < right:
                            self.search(q,node.right)

    def get_ordering(self):
        indices = []
        self.root.get_ordering(indices)
        return indices

    def set_ordering(self,index_map):
        self.root.set_ordering(index_map)

In [65]:
tree = MIPTree(item_vectors,N0=20,reorder=True)

/home/ubuntu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:96: DeprecationWarning: Assigning the 'data' attribute is an inherently unsafe operation and will be removed in the future.


In [70]:
user_vector = model.user_embeddings[0].shape

In [68]:
nn, scanned = tree.match(user_vector, 10)

In [73]:
def approx_product_search(user_id, tree, n=10):
    user_vector = model.user_embeddings[user_id]
    nn, scanned = tree.match(user_vector, n)
    print('Approxiamtely recommended for user id %s : \n' % str(user_id))
    titles = [movielens['item_labels'][i[0]] for i in nn]
    print("\n".join(titles))

In [76]:
approx_product_search(32, tree, n=10)

Approxiamtely recommended for user id 32 : 

Little Big League (1994)
Tie Me Up! Tie Me Down! (1990)
Boys (1996)
Gaslight (1944)
Fathers' Day (1997)
Seven (Se7en) (1995)
8 1/2 (1963)
Home for the Holidays (1995)
Juror, The (1996)
Crossfire (1947)
